## Phidget4-anywidget_v2

November 5. Updated November 11. Version 2 on Nov 2.

A test to connect four Phidgets to gauges, and update correctly.

Includes code to gracefully disconnect the Phidgets whenever the kernel shuts down or the notebook is closed. (At least I think it does.)

For version 2, organize the code into functions that get called by the UI. Also allow messages to be sent from Python to read the gauges, open and close channels. 

In [1]:
import anywidget
import traitlets


In [10]:
class FourPhidgets(anywidget.AnyWidget):
    _esm = """
    import  "https://unpkg.com/phidget22/browser/phidget22";

    export function render({ model, el }) {
      let usbconn = 0;
      let tSensor = 0;
      let hSensor = 0;
      let mSensor = 0;
      let lSensor = 0;
      let errorCode = 0;
      let getTemperature = () => model.get("temperature");
      let getHumidity = () => model.get("humidity");
      let getMoisture = () => model.get("moisture");
      let getLuminance = () => model.get("luminance");
      function openUSB() {
        usbconn = new phidget22.USBConnection();    
        usbconn.connect().then(() => {
          usbconn.requestWebUSBDeviceAccess();
        }).catch(err => {
          // usbconn.delete();
          console.log('Error connecting to USB ' + err);
        });
      };
      async function openDevices() {
        tSensor = new phidget22.TemperatureSensor();
        tSensor.open(2000);
        hSensor = new phidget22.HumiditySensor();
        hSensor.open(2000);
        mSensor = new phidget22.VoltageRatioInput();
        mSensor.open(2000);
        lSensor = new phidget22.LightSensor();
        lSensor.open(2000);
      };
      function readDevices() {
        let t0 = 0;
        let h0 = 0;
        let m0 = 0;
        let l0 = 0;
        if (tSensor.attached) {t0 = tSensor.temperature; }       
        if (hSensor.attached) {h0 = hSensor.humidity; }       
        if (mSensor.attached) {m0 = mSensor.voltageRatio; }       
        if (lSensor.attached) {l0 = lSensor.illuminance; }       
        model.set('temperature', t0 );
        model.set('humidity', h0);
        model.set('moisture', m0);
        model.set('luminance', l0);
        model.save_changes();
      };
      async function closeDevices() {
        try {await tSensor.close();} catch {}
        try {await hSensor.close();} catch {}
        try {await mSensor.close();} catch {}
        try {await lSensor.close();} catch {}
      };
      async function closeUSB() {
        try {await usbconn.close();} catch {}
        try {await usbconn.delete();} catch {}
      };
      model.on('msg:custom', msg => {
         console.log(`new message: ${JSON.stringify(msg)}`);
         if (msg == 'open') {
            console.log('open called');
            openUSB();
            openDevices();
         }
         if (msg == 'read') {
            console.log('read called');
            readDevices();
         }
         if (msg == 'close') {
            console.log('close called');
            closeDevices();
            closeUSB();
         }
      });

      let buttonUSB = document.createElement("button");
      buttonUSB.classList.add("ph-button");
      buttonUSB.innerHTML = `Click to open USB`;
      let buttonOpen = document.createElement("button");
      buttonOpen.classList.add("ph-button");
      buttonOpen.innerHTML = `Click to connect sensors`;
      let buttonRead = document.createElement("button");
      buttonRead.classList.add("ph-button");
      buttonRead.innerHTML = `Click to read data`;
      let buttonClose = document.createElement("button");
      buttonClose.classList.add("ph-button");
      buttonClose.innerHTML = `Click to disconnect`;
      let textTemp = document.createElement("textarea");
      textTemp.classList.add("ph-text");
      textTemp.value = 'Temperature';
      let textHum = document.createElement("textarea");
      textHum.classList.add("ph-text");
      textHum.value = 'Humidity';
      let textMoist = document.createElement("textarea");
      textMoist.classList.add("ph-text");
      textMoist.value = 'Moisture';
      let textLite = document.createElement("textarea");
      textLite.classList.add("ph-text");
      textLite.value = 'Light level';
      buttonUSB.addEventListener("click", openUSB );
      buttonOpen.addEventListener("click", openDevices );
      buttonRead.addEventListener("click", readDevices );
      buttonClose.addEventListener("click", async () => {
        closeDevices();
        closeUSB();
        });
      model.on("change:temperature", () => {
        textTemp.value = `Temperature is ${getTemperature()}`;
      });
      model.on("change:humidity", () => {
        textHum.value = `Humidity is ${getHumidity()}`;
      });
      model.on("change:moisture", () => {
        textMoist.value = `Moisture is ${getMoisture()}`;
     });
      model.on("change:luminance", () => {
        textLite.value = `Luminance is ${getLuminance()}`;
      });
      el.appendChild(buttonUSB);
      el.appendChild(buttonOpen);
      el.appendChild(buttonRead);
      el.appendChild(buttonClose);
      el.appendChild(textTemp);
      el.appendChild(textHum);
      el.appendChild(textMoist);
      el.appendChild(textLite);
      // we include a return function to close the Phidget when the notebook is close
      return async () => {
        try {await tSensor.close();} catch {}
        try {await hSensor.close();} catch {}
        try {await mSensor.close();} catch {}
        try {await lSensor.close();} catch {}
        try {await usbconn.close();} catch {}
        try {await usbconn.delete();} catch {}
    };
    }
    """
    _css="""
    .ph-button {color: white; 
                background-color: var(--md-blue-700); 
                border-radius: 8px; 
                font-size: 24px; 
                display: block;
                padding: 15px 32px;}
    .ph-button:hover { background-color: var(--md-blue-900); }
    """
    temperature = traitlets.Float(0).tag(sync=True)
    humidity = traitlets.Float(0).tag(sync=True)
    moisture = traitlets.Float(0).tag(sync=True)
    luminance = traitlets.Float(0).tag(sync=True)

fp = FourPhidgets()
fp.temperature = 40
fp.humidity = 41
fp.moisture = 42
fp.luminance = 43

In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets


In [12]:
# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 10000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.35
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

In [13]:
def updateTemperature(change):
    gauges.data[0]['value'] = change.new
    
def updateHumidity(change):
    gauges.data[1]['value'] = change.new
    
def updateMoisture(change):
    gauges.data[2]['value'] = change.new
    
def updateLuminance(change):
    gauges.data[3]['value'] = change.new
    

In [14]:
fp.observe(updateTemperature, names=["temperature"])
fp.observe(updateHumidity, names=["humidity"])
fp.observe(updateMoisture, names=["moisture"])
fp.observe(updateLuminance, names=["luminance"])


In [15]:
gauges


FigureWidget({
    'data': [{'domain': {'x': [0.0, 0.45], 'y': [0.675, 1.0]},
              'gauge': {'axis': {'range': [10, 40]}},
              'mode': 'gauge+number',
              'title': {'text': 'Temperature'},
              'type': 'indicator',
              'uid': 'a36a2330-457d-4b7d-ace2-30b7580227c4',
              'value': 20},
             {'domain': {'x': [0.55, 1.0], 'y': [0.675, 1.0]},
              'gauge': {'axis': {'range': [0, 100]}},
              'mode': 'gauge+number',
              'title': {'text': 'Humidity'},
              'type': 'indicator',
              'uid': '125f9b06-b3a4-4308-99c3-9f90b28b8f66',
              'value': 40},
             {'domain': {'x': [0.0, 0.45], 'y': [0.0, 0.325]},
              'gauge': {'axis': {'range': [0, 1.0]}},
              'mode': 'gauge+number',
              'title': {'text': 'Moisture'},
              'type': 'indicator',
              'uid': '52c57260-7dce-4268-906c-6d0d465c1ceb',
              'value': 0.5},
         

In [16]:
fp



FourPhidgets(humidity=41.0, luminance=43.0, moisture=42.0, temperature=40.0)

In [4]:
fp.send("open")

In [7]:
fp.send("read")

In [8]:
fp.send("close")